## 作業目標: 使用Jieba進行各項的斷詞操作

這份作業我們會使用中文飯店評論資料集來作為斷詞練習。 [資料集:ChnSentiCorp_htl_all](https://github.com/SophonPlus/ChineseNlpCorpus)

### 讀入文本資料

In [1]:
import pandas as pd

###<your code>###
#with open('./ChnSentiCorp_htl_all.csv', 'r') as f
#    f.read()
pd_corpus = pd.read_csv('./ChnSentiCorp_htl_all.csv')  
pd_corpus.head(5)

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [2]:
pd_corpus.shape

(7766, 2)

In [3]:
#確認所有留言,正評價(label=1)與負評價數量(label=0)
###<your code>###
#print(pd_corpus.loc[pd_corpus['label'] == 1, :].count())
#print(pd_corpus.loc[pd_corpus['label'] == 0, :].count())
pd_positive = pd_corpus.loc[pd_corpus['label'] == 1, :]
pd_negative = pd_corpus.loc[pd_corpus['label'] == 0, :]
print(f'Total: {len(pd_corpus)}, Positive: {len(pd_positive)}, Negative: {len(pd_negative)}')

Total: 7766, Positive: 5322, Negative: 2444


In [4]:
#移除缺失值
###<your code>###
pd_corpus.dropna(inplace=True) # 在原數據集上操作

### 建構結巴斷詞Function

建構可將讀入的Pandas DataFrame的文本資料，外加一欄名為`cut`的review欄位斷詞結果

In [5]:
import jieba

In [6]:
class JiebaCutingClass(object):
    '''Class to use jeiba to parse corpus from dataframe and cut the corpus
    
    Parameters
    -----------
    key_to_cut: str
        the dataframe key to parse the sentence for jieba cutting
    dic: str
        the dictionary to use for jieba, default is None (use default dictionary)
    userdict: str
        the user defined dictionary to use for jieba, default is None
    '''
    
    def __init__(self, key_to_cut:str, dic:str=None, userdict:str=None):
        
        if dic is not None:
            jieba.set_dictionary(dic) #使用繁體詞庫
        
        if userdict is not None:
            jieba.load_userdict(userdict) #載入自定義詞庫
        
        self.key_to_cut = key_to_cut
        
        #將paddle 功能開啟
        ###<your code>###
        jieba.enable_paddle() #啟用paddle模式
        
    @staticmethod
    def cut_single_sentence(sentence, use_paddle=False, use_full=False, use_search=False):
        
        if use_search:
            ###<your code>### 搜尋引擎模式
            out = jieba.cut_for_search(sentence) 
        else:
            ###<your code>### paddle模式, 精確模式/全模式
            out= jieba.cut(sentence, use_paddle=use_paddle, use_full=use_full) 
        
        return out
            
    
    def cut_corpus(self, corpus: pd.DataFrame, mode: str) -> pd.DataFrame:
        '''Method to read and cut sentence from dataframe and append another column named cut
        
        Paremeters
        --------------
        corpus: pd.DataFrame
            Input corpus in dataframe
        mode: str
            Jieba mode to be used
        
        Return
        ----------------
        out: pd.Dataframe
            Output corpus in dataframe
        '''
        
        # checking valid mode
        if mode not in ['paddle', 'full', 'precise', 'search']:
            raise TypeError(f'only support `paddle`, `full`, `precise`, and `search` mode, but get {mode}')
            
        # cut the corpus based on mode
        if mode == 'paddle':
            ###<your code>###
            out = self._paddle_cut(corpus)
        elif mode == 'full':
            ###<your code>###
            out = self._full_cut(corpus)
        elif mode == 'precise':
            ###<your code>###
            out = self._precise_cut(corpus)
        elif mode == 'search':
            ###<your code>###
            out = self._search_cut(corpus)

        return out
    
    def _paddle_cut(self, corpus):
        '''paddle mode
        '''
        
        out = []
        ###<your code>###
        jieba.enable_paddle() #啟用paddle模式
        #print(corpus)
        for corpu in corpus[self.key_to_cut]:
            words = jieba.cut(corpu, use_paddle=True)
            out.append([word for word in words])
        corpus['cut'] = out
        
        return corpus
    
    def _full_cut(self, corpus):
        '''full mode
        '''
        
        out = []
        ###<your code>###
        for corpu in corpus[self.key_to_cut]:
            words = jieba.cut(corpu,cut_all=True)
            out.append([word for word in words])
        corpus['cut'] = out
                
        return corpus
    
    def _precise_cut(self, corpus):
        '''precise mode
        '''
        
        out = []
        ###<your code>###
        for corpu in corpus[self.key_to_cut]:
            words = jieba.cut(corpu,cut_all=False)
            out.append([word for word in words])
        corpus['cut'] = out
        
        return corpus
    
    def _search_cut(self, corpus):
        '''search mode
        '''
        
        out = []
        ###<your code>###
        for corpu in corpus[self.key_to_cut]:
            words = jieba.cut_for_search(corpu)
            out.append([word for word in words])
        corpus['cut'] = out
        
        return corpus

### 使用建構好的斷詞物件對文本進行斷詞

In [12]:
### 請使用精確模式與預設字典對文本進行斷詞

###<your code>###
###<your code>### #為了避免處理時間過久, 這裡我們只使用前50個進行斷詞
jieba_cut = JiebaCutingClass(key_to_cut='review')
# <>. 'paddle', 'full', 'precise', 'search'
#pd_cut = jieba_cut.cut_corpus(pd_corpus.loc[:50, :], mode='paddle') #為了避免處理時間過久, 這裡我們只使用前50個進行斷詞
#pd_cut = jieba_cut.cut_corpus(pd_corpus.loc[:50, :], mode='full')
#pd_cut = jieba_cut.cut_corpus(pd_corpus.loc[:50, :], mode='precise')
pd_cut = jieba_cut.cut_corpus(pd_corpus, mode='search')
pd_cut.head()

Paddle enabled successfully......


,label,review,cut
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较...","[距离, 川沙, 公路, 较近, ,, 但是, 公交, 指示, 不, 对, ,, 如果, 是..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!,"[商务, 大床, 房, ，, 房间, 很大, ，, 床有, 2M, 宽, ，, 整体, 感觉..."
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。,"[早餐, 太, 差, ，, 无论, 去, 多少, 人, ，, 那边, 也, 不加, 食品, ..."
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...,"[宾馆, 在, 小, 街道, 上, ，, 不大, 大好, 不大好, 找, ，, 但, 还好,..."
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风","[CBD, 中心, ,, 周围, 什么, 没什么, 店铺, ,, 说, 5, 星, 有点, ..."


In [13]:
pd_cut

,label,review,cut
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较...","[距离, 川沙, 公路, 较近, ,, 但是, 公交, 指示, 不, 对, ,, 如果, 是..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!,"[商务, 大床, 房, ，, 房间, 很大, ，, 床有, 2M, 宽, ，, 整体, 感觉..."
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。,"[早餐, 太, 差, ，, 无论, 去, 多少, 人, ，, 那边, 也, 不加, 食品, ..."
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...,"[宾馆, 在, 小, 街道, 上, ，, 不大, 大好, 不大好, 找, ，, 但, 还好,..."
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风","[CBD, 中心, ,, 周围, 什么, 没什么, 店铺, ,, 说, 5, 星, 有点, ..."
5,1,总的来说，这样的酒店配这样的价格还算可以，希望他赶快装修，给我的客人留些好的印象,"[来说, 总的来说, ，, 这样, 的, 酒店, 配, 这样, 的, 价格, 还, 算, 可..."
6,1,价格比比较不错的酒店。这次免费升级了，感谢前台服务员。房子还好，地毯是新的，比上次的好些。早...,"[价格, 价格比, 比较, 不错, 的, 酒店, 。, 这次, 免费, 升级, 了, ，, ..."
7,1,不错，在同等档次酒店中应该是值得推荐的！,"[不错, ，, 在, 同等, 档次, 酒店, 中, 应该, 是, 值得, 推荐, 的, ！]"
8,1,入住丽晶，感觉很好。因为是新酒店，的确有淡淡的油漆味，房间内较新。房间大小合适，卫生间设备齐...,"[入住, 丽晶, ，, 感觉, 很, 好, 。, 因为, 是, 新, 酒店, ，, 的确, ..."
9,1,1。酒店比较新，装潢和设施还不错，只是房间有些油漆味。2。早餐还可以，只是品种不是很多。3。...,"[1, 。, 酒店, 比较, 新, ，, 装潢, 和, 设施, 还, 不错, ，, 只是, ..."


### 載入繁中字典為指定的字詞進行斷詞

In [16]:
test_string = '我愛cupoy自然語言處理馬拉松課程'

jieba.set_dictionary('dict.txt.big') # 載入繁體詞庫:
###<your code>###
out_string = jieba.cut(test_string,use_paddle=True) ###<your code>### #paddle 模式
print(f'Paddle模式: {[string for string in out_string]}')

out_string = jieba.cut(test_string,cut_all=True) ###<your code>### #全模式
print(f'全模式: {[string for string in out_string]}')

out_string = jieba.cut_for_search(test_string) ###<your code>### #搜尋模式
print(f'搜尋模式: {[string for string in out_string]}')

out_string = jieba.cut(test_string, cut_all=False) ###<your code>### #精確模式
print(f'精確模式: {[string for string in out_string]}')

Building prefix dict from D:\Work\Product\Python\eBook\第一屆_NLP 自然語言機器學習馬拉松\Training\Part1\Homework\dict.txt.big ...
Loading model from cache C:\Users\angus.tu\AppData\Local\Temp\jieba.u38afb3e2effc8eb2cdf4695c155ff190.cache


Paddle模式: ['我', '愛', 'cupoy', '自然', '語言', '處理', '馬拉松', '課程']


Loading model cost 2.044 seconds.
Prefix dict has been built successfully.


全模式: ['我', '愛', 'cupoy', '自然', '自然語言', '語言', '處理', '馬拉', '馬拉松', '課程']
搜尋模式: ['我', '愛', 'cupoy', '自然', '語言', '自然語言', '處理', '馬拉', '馬拉松', '課程']
精確模式: ['我', '愛', 'cupoy', '自然語言', '處理', '馬拉松', '課程']
